# PW4: Genetic algorithms for optimization
*Alexandra Korukova, Samuel Mayor*  
*HEIG-VD - MLG - 30.05.2019*

In [22]:
from scipy import sin, cos, tan, arctan, arctan2, arccos, pi
import numpy as np

In [17]:
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]

LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

In [18]:
def spherical_distance(lat1, long1, lat2, long2):
    phi1 = 0.5*pi - lat1
    phi2 = 0.5*pi - lat2
    r = 0.5*(6378137 + 6356752) # mean radius in meters
    t = sin(phi1)*sin(phi2)*cos(long1-long2) + cos(phi1)*cos(phi2)
    return r * arccos(t)

In [24]:
# creates latitude/longitude tuples for each city
coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))

# calculate distances between each city pairs
distances = np.zeros((14, 14))
for i,c1 in enumerate(coords):
    for j,c2 in enumerate(coords):
        distances[i, j] = spherical_distance(c1[0], c1[1], c2[0], c2[1])
distances

/home/etnarion/miniconda3/envs/MLGPW4/lib/python2.7/site-packages/ipykernel_launcher.py:10: ComplexWarning: Casting complex values to real discards the imaginary part
  # Remove the CWD from sys.path while we load stuff.


array([[0.00000000e+00, 7.17150641e+06, 1.28933471e+07, 1.22302394e+07,
        1.23996208e+07, 4.80427620e+06, 1.46581084e+07, 4.65800628e+06,
        6.23724317e+06, 1.45953468e+07, 5.52209152e+06, 8.39826812e+06,
        1.47845984e+07, 1.45743362e+07],
       [7.17150641e+06, 0.00000000e+00, 1.39198265e+07, 5.99794107e+06,
        5.81350831e+06, 1.01435940e+07, 1.51195135e+07, 5.30484644e+06,
        1.13027731e+07, 1.43721526e+07, 9.85390436e+06, 1.03208588e+07,
        8.82140471e+06, 1.69381258e+07],
       [1.28933471e+07, 1.39198265e+07, 0.00000000e+00, 1.38568125e+07,
        9.44182292e+06, 8.10034451e+06, 2.05301988e+06, 1.74732260e+07,
        1.38030859e+07, 1.71667068e+06, 1.51295470e+07, 4.90952630e+06,
        6.89164115e+06, 3.53226257e+06],
       [1.22302394e+07, 5.99794107e+06, 1.38568125e+07, 9.48823169e-02,
        5.52405292e+06, 1.61414898e+07, 1.28169422e+07, 7.95189562e+06,
        1.17910078e+07, 1.25522822e+07, 1.07728677e+07, 1.47000406e+07,
        7.174